In [ ]:
%%init_spark
launcher.master="yarn"
launcher.num_executors=6
launcher.executor_memory="6000m"




In [ ]:
import org.apache.hadoop.mapreduce.lib.input.TextInputFormat
import org.apache.hadoop.io.LongWritable
import org.apache.hadoop.io.Text
val conf = sc.hadoopConfiguration
conf.set("textinputformat.record.delimiter", "\n\n")



val wikilinks=sc.newAPIHadoopFile("/hadoop-user/data/wikilinks.txt", classOf[TextInputFormat],classOf[LongWritable], classOf[Text], conf)
.map{case(offset,value)=>value.toString.trim()}
.filter(value=>value.length!=0)


def extractURLandMentions(rddelement:String):(String,Array[String])={
    val splits=rddelement.split("\n")
    var url=""
//var Mentions = collection.mutable.ArrayBuffer
    var Mentions= Array[String]()
    
for(line<-splits)
{
    if(line.startsWith("URL"))
       {val splits=line.split("\t");
      if (splits.length>=2) url=(splits(1))}
    else if(line.startsWith("MENTION"))
    {
        val splits=line.split("\t");
       if(splits.length>=3) 
        Mentions:+=(splits(3))
    }

}
   return(url,Mentions)
}
val logfile=wikilinks.map(rddelement=>extractURLandMentions(rddelement)).flatMapValues(rddelement=>rddelement)

logfile.take(3).foreach(println)
//The symmetric pair is 0 . So I have commented the code.
//val URLA=logfile.map{case(url,mention)=>(mention,url)}
//val joined= logfile.join(URLA)
//joined.take(3).foreach(println)



//finding the number of times each URL has been mentioned by another URL
val counts=logfile.map{case(url1,url2)=>(url2,1)}.reduceByKey((url1,url2)=>url1+url2)

counts.take(10).foreach(println)










In [ ]:


//logfile.take(10).foreach(println)